In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import erf
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
#%matplotlib tk

In [2]:
result = pd.read_csv("/home/lin/git/summerProject/Heston++onSPXonly_num_estar15_100run")

In [3]:
result[['time', 'iteration', 'pv', 'jac', 'lin_sys', 'e0', 'e*', 'Je', 'Dp', 'count','count2', 'count7']].describe()

,time,iteration,pv,jac,lin_sys,e0,e*,Je,Dp,count,count2,count7
count,100.000000,100.000000,100.000000,100.00000,100.000000,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,100.000000,100.0,100.000000
mean,3.174657,410.120000,986.680000,41.11000,410.140000,1.178161e+05,3.627979e-01,4.831204e+00,1.954844e-04,1.330000,0.0,0.330000
std,1.055139,133.025651,317.373784,13.18623,132.987811,8.309394e+05,2.373026e+00,3.658622e+01,9.444786e-04,0.876805,0.0,0.876805
min,0.756609,95.000000,236.000000,10.00000,95.000000,9.874506e-04,8.939360e-19,5.343369e-12,0.000000e+00,1.000000,0.0,0.000000
25%,2.405413,291.000000,708.500000,30.00000,291.000000,3.916839e-02,8.755823e-16,3.052796e-07,1.037326e-07,1.000000,0.0,0.000000
50%,3.682545,500.000000,1196.500000,50.00000,500.000000,1.247562e-01,1.588109e-12,3.006577e-06,9.785607e-07,1.000000,0.0,0.000000
75%,3.909041,500.000000,1201.000000,50.00000,500.000000,2.554219e-01,1.175978e-09,3.623202e-05,1.111475e-05,1.000000,0.0,0.000000
max,4.522619,500.000000,1229.000000,52.00000,500.000000,7.953150e+06,2.097765e+01,3.564130e+02,8.229071e-03,6.000000,0.0,5.000000


In [4]:
result['stopped_by'].unique()
#This show the reasons for stopping:
#1 - stopped by small gradient J^T e
#2 - stopped by small Dp
#3 - stopped by itmax
#4 - singular matrix. Restart from current p with increased \mu
#5 - no further error reduction is possible. Restart with increased mu
#6 - stopped by small ||e||_2
#7 - stopped by invalid (i.e. NaN or Inf) "func" values; a user error

array([6, 3, 5, 1])

In [5]:
#number of solved cases, allowing restart when stopped by 7 or 2
solved = result[result['stopped_by'] == 6]
solved.count().unique()

array([31])

In [6]:
#percentage of solved
solved.count().unique()/result.count().unique()

array([0.31])

In [7]:
#number of solved cases at the first try
solved_first_time = solved[solved['count'] == 1]
solved_first_time.count().unique()

array([27])

In [8]:
#number of cases stopped by small Dp (update of parameter)
DP = result[result['stopped_by'] == 2]
DP.count().unique()

array([0])

In [9]:
#number of cases that stopped by max iteration
itmax = result[result['stopped_by'] == 3]
itmax.count().unique()

array([61])

In [10]:
#number of cases that stopped by mall gradient J^T e
smalJac = result[result['stopped_by'] == 1]
smalJac.count().unique()

array([6])

In [11]:
#average time for all solved cases
solved['time'].describe()['mean']

2.2267183225806453

In [12]:
#average time for cases solved without restart
solved_first_time['time'].describe()['mean']

2.1852084814814816

In [13]:
#Average number of iteration for solved cases
solved_first_time['iteration'].describe()['mean']

282.18518518518516

In [14]:
#Average number of iteration for all solved cases
solved['iteration'].describe()['mean']

288.4193548387097

In [15]:
#All solved cases
solved[['Ok', 'Rk', 'Ovbar', 'Rvbar','Ov0', 'Rv0', 'Orho', 'Rrho', 'Osigma', 'Rsigma', 'OI1', 'RI1', 'OI2', 'RI2', 'stopped_by', 'e*']]

,Ok,Rk,Ovbar,Rvbar,Ov0,Rv0,Orho,Rrho,Osigma,Rsigma,OI1,RI1,OI2,RI2,stopped_by,e*
0,1.091921,1.091920,0.462785,0.462785,0.247063,0.247063,-0.643092,-0.643092,0.891224,0.891224,0.000519,0.000519,0.000035,0.000035,6,3.921817e-18
10,0.958627,0.957368,0.622201,0.621246,0.765101,0.765293,-0.702485,-0.702352,0.620133,0.620087,0.000605,0.000598,0.000322,0.000319,6,9.281671e-16
15,3.594963,3.593365,0.801790,0.802586,0.924518,0.926012,-0.164413,-0.164282,0.743140,0.742494,0.000308,0.000250,0.000821,0.000769,6,9.010909e-16
16,1.246245,1.246674,0.193441,0.193782,0.645925,0.645794,-0.859132,-0.859326,0.608952,0.608941,0.000013,0.000018,0.000718,0.000722,6,4.175180e-16
22,3.078636,3.079323,0.231973,0.232234,0.926979,0.926913,-0.493457,-0.493473,0.774382,0.774421,0.000158,0.000160,0.000149,0.000149,6,7.506368e-16
23,0.641321,0.641317,0.519992,0.519993,0.299478,0.299478,-0.569490,-0.569490,0.780706,0.780706,0.000940,0.000940,0.000580,0.000580,6,3.871764e-18
26,4.643714,4.643733,0.514879,0.514878,0.287721,0.287720,-0.766918,-0.766919,0.621157,0.621157,0.000458,0.000458,0.000156,0.000156,6,4.251595e-18
27,0.833775,0.832326,0.559786,0.559564,0.430229,0.430297,-0.334454,-0.334435,0.371819,0.371773,0.000592,0.000589,0.000464,0.000463,6,9.997279e-16
31,2.773030,2.772765,0.156626,0.156558,0.652009,0.652014,-0.409366,-0.409366,0.797998,0.797987,0.000124,0.000123,0.000648,0.000648,6,3.764429e-16
33,4.067321,4.067511,0.242513,0.242524,0.392577,0.392565,-0.842797,-0.842822,0.604904,0.604907,0.000356,0.000356,0.000198,0.000198,6,8.522514e-16


In [16]:
#next 7 blocks show the average distance between initial guess and real parameter
disIk = np.abs(solved['Ik'] - solved['Rk']).mean()
disIk

1.1725322542588388

In [17]:
disIvbar = np.abs(solved['Ivbar'] - solved['Rvbar']).mean()
disIvbar

0.23527842205340357

In [18]:
disIv0 = np.abs(solved['Iv0'] - solved['Rv0']).mean()
disIv0

0.24626006432452363

In [19]:
disIsigma = np.abs(solved['Isigma'] - solved['Rsigma']).mean()
disIsigma

0.2563367856929806

In [20]:
disIrho = np.abs(solved['Irho'] - solved['Rrho']).mean()
disIrho

0.20268818744517678

In [21]:
disII1 = np.abs(solved['II1'] - solved['RI1']).mean()
disII1

0.0003205367553734801

In [22]:
disII2 = np.abs(solved['II2'] - solved['RI2']).mean()
disII2

0.00026621163592874643

In [23]:
#next 7 blocks show the average distance between optimal found and real parameter
disOk = np.abs(solved['Ok'] - solved['Rk']).mean()
disOk

0.0005023735753890522

In [24]:
disOvbar = np.abs(solved['Ovbar'] - solved['Rvbar']).mean()
disOvbar

0.00019952606063884123

In [25]:
disOsigma = np.abs(solved['Osigma'] - solved['Rsigma']).mean()
disOsigma

6.023291210299137e-05

In [26]:
disOrho = np.abs(solved['Orho'] - solved['Rrho']).mean()
disOrho

3.805521613678957e-05

In [27]:
disOv0 = np.abs(solved['Ov0'] - solved['Rv0']).mean()
disOv0

0.00015206417543297743

In [28]:
disOI1 = np.abs(solved['OI1'] - solved['RI1']).mean()
disOI1

5.757732174755785e-06

In [29]:
disOI2 = np.abs(solved['OI2'] - solved['RI2']).mean()
disOI2

4.884728898739867e-06